In [25]:
import cv2
import numpy as np
import os

In [26]:
def save_image(image, filename):
    """
    Save the image to a file.

    Parameters:
        image (numpy.ndarray): The image to save.
        filename (str): The filename to save the image as.
    """
    cv2.imwrite(filename, image)

In [27]:
def fill_region_and_export(binary_image, x, y, output_dir):
    """
    Highlights the (x, y) point and fills the region inside the contour containing the point.
    Saves intermediate results as images.

    Parameters:
        binary_image (numpy.ndarray): Binary image with black background and white contours (2D array of 0s and 255s).
        x (int): X-coordinate in the image.
        y (int): Y-coordinate in the image.
        output_dir (str): Directory where the output images will be saved.

    Returns:
        None
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save the original image
    save_image(binary_image, os.path.join(output_dir, "stage_1_original.png"))

    # Ensure the input is a binary image
    if len(binary_image.shape) != 2 or binary_image.dtype != np.uint8:
        raise ValueError("Input must be a binary image (2D array of dtype uint8).")

    # Check if the given pixel is inside a contour
    if binary_image[y, x] == 255:
        raise ValueError("The given point is in the contout")

    # Find all contours
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour containing the point
    contour_found = False
    for contour in contours:
        if cv2.pointPolygonTest(contour, (x, y), measureDist=False) >= 0:
            contour_found = True
            # Create a mask for the specific contour
            mask = np.zeros_like(binary_image, dtype=np.uint8)
            cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
            save_image(mask, os.path.join(output_dir, "stage_2_filled_region.png"))
            break

    if not contour_found:
        raise ValueError("No contour found containing the given point.")

    # Apply the mask to create the output
    output_image = mask.copy()

    # Highlight the point on the output image
    output_image = cv2.circle(output_image, (x, y), radius=3, color=128, thickness=-1)
    save_image(output_image, os.path.join(output_dir, "stage_3_final_output.png"))

In [28]:
if __name__ == "__main__":
    # Create a complex binary image with a black background and white contours
    binary_image = np.zeros((400, 400), dtype=np.uint8)

    # Draw multiple white contours for complexity
    cv2.rectangle(binary_image, (50, 50), (150, 150), 255, thickness=2)
    cv2.circle(binary_image, (200, 200), 50, 255, thickness=2)
    cv2.ellipse(binary_image, (300, 300), (60, 40), 0, 0, 360, 255, thickness=2)
    cv2.line(binary_image, (50, 300), (150, 350), 255, thickness=2)

    # Coordinates for the point (inside the rectangle)
    x, y = 120, 120

    # Output directory
    output_directory = "output_images"

    # Call the function
    fill_region_and_export(binary_image, x, y, output_directory)

    print(f"Images saved in the directory: {output_directory}")

Images saved in the directory: output_images
